# Unit-Tests

Mit Unit-Tests kann die Funktionalität einer Python-Anwendung automatisiert getestet werden.
Gleichzeitig sind sie für später eine Dokumentation (wie können die einzelnen Funktionen / Methoden aufgerufen werden, was geben sie zurück).

Tests werden üblicherweise in einem `test` / `tests` Ordner untergebracht.

```
projekt/
├── README.md
├── modul/
|   ├── __init__.py
|   └── ...
└── test/
    ├── test_a.py
    ├── test_b.py
    └── ...
```

Und dann mit einem Testrunner kollektiv / parallel gestestet.
Die Python Standardbibliothek kommt bereits mit einem test framework / Testrunner [unittest](https://docs.python.org/3/library/unittest.html).
Es hat sich jedoch in den meisten Projekten das Externe Framework [pytest](https://docs.pytest.org/en/7.3.x/) als Standard durchgesetzt.

## Das *py.test* Framework:

Das *pytest*-Modul ist ähnlich dem [unittest](https://docs.python.org/3/library/unittest.html)-Modul. Es muss mit `pip` installiert werden:

In [ ]:
!pip install pytest --break-system-packages

In *pytest* beginnt jede Methode mit `test_`. Mit dem `assert` Schlüsselwort wird der erwartete Wert mit dem aktuellen Wert verglichen.

### Beispiel: `SurveyResponse` Klasse

Um direkt innerhalb dieses Notebooks tests laufen lassen zu können, verwenden wir die `ipython_pytest`-Erweiterung.
Normalerweise würde aber der folgende Test in ein File im Ordner `test` / `tests` geschrieben werden und mittels
```sh
pytest
```
im Projektordner getestet werden.

In [ ]:
%load_ext ipython_pytest

In [ ]:
%%pytest

import pytest

from survey_response import SurveyResponse

BODY = """First name: Chuck
name: Norris
mail: chucknorris@roundhouse.gov
lecture: Roundhouse Kicks
"""

RESPONSE = SurveyResponse(
    first_name="Chuck",
    name="Norris",
    lecture="Roundhouse Kicks",
    mail="chucknorris@roundhouse.gov"
)

# unit tests using pyest:

def test_survey_response_can_load_simple_response():
    expected_response = RESPONSE
    assert SurveyResponse.from_body(BODY) == expected_response

def test_survey_response_raises_error_for_invalid_field():
    body = BODY + "comment: Chucknorris counted to ∞ - twice!\n"
    with pytest.raises(TypeError):
        SurveyResponse.from_body(body)

def test_servey_response_creates_correct_csv_line():
    csv_line = RESPONSE.to_csv()
    expected_csv_line = "; Chuck; Norris; chucknorris@roundhouse.gov; Roundhouse Kicks; 'Nein'; "
    assert csv_line == expected_csv_line


Stellt `assert` einen Fehler fest, wird eine ausführliche Meldung ausgegeben.

Üblicherweise befindet die Funktionlität in einem Modul (z.B. *point.py*), während für die Tests ein zweites Modul erzeugt wird (z.B. *test_point.py*). Mit dem Befehl `pytest` werden die Module *test_???.py* im aktuellen Verzeichnis ausgefüht, d.h. es werden alle Funktionen in solchen Modulen, welche mit `test_` beginnen, aufgerufen.

Wird *pytest* auf der Kommandozeile aufgerufen, können Flags angefügt werden (siehe [pytest - Usage and Invocations](https://docs.pytest.org/en/stable/usage.html)):
- `pytest -x`: nach dem ersten Fehler stoppen
- `pytest --lf`: den Test mit dem letzten Fehler aufrufen
- `pytest --pdb`: in den Python-Debugger (PDB) wechseln, wenn ein Fehler gefunden wird



### Arbeiten mit *pytest fixtures*

Im Test der `SurveyResponse`-Klasse wurden in den einzelnen Tests die Variablen `BODY` und `RESPONSE` mehrfach verwendet.
Um unabsichtliches Überschreiben dieser dieser toplevel Objekte durch Tests zu vermeiden, können *Fixture*s verwendet werden. Eine Fixture ist eine Methode, welche mit `@pytest.fixture` annotiert ist und verwendet werden kann, um die Erzeugung von Testobjekten zu vereinfachen.  
Die Fixture wird den Testmethoden als Parameter übergeben und kennzeichnet so explizit den Gebrauch dieses Parameters im Test.

In [ ]:
%%pytest

import pytest

from survey_response import SurveyResponse

@pytest.fixture
def given_body():
    return """First name: Chuck
name: Norris
mail: chucknorris@roundhouse.gov
lecture: Roundhouse Kicks
"""

@pytest.fixture
def response():
    return SurveyResponse(
        first_name="Chuck",
        name="Norris",
        lecture="Roundhouse Kicks",
        mail="chucknorris@roundhouse.gov"
    )


def test_survey_response_can_load_simple_response(given_body, response):
    expected_response = response
    assert SurveyResponse.from_body(given_body) == expected_response

def test_survey_response_raises_error_for_invalid_field(given_body):
    given_body += "comment: Chucknorris counted to ∞ - twice!\n"
    with pytest.raises(TypeError):
        SurveyResponse.from_body(body)

def test_servey_response_creates_correct_csv_line(response):
    csv_line = response.to_csv()
    expected_csv_line = "; Chuck; Norris; chucknorris@roundhouse.gov; Roundhouse Kicks; 'Nein'; "
    assert csv_line == expected_csv_line


Fixtures eignen sich speziell, wenn Ressourcen zur Verfügung gestellt werden müssen, deren Erzeugung zeitaufwendig ist (z.B. DB- oder Online-Verbindungen). In diesem Fall kann die Fixture mit einem Scope versehen werden. Folgende Scopes sind möglich: 
- *function*: die Fixture wird für jede Testfunktion erstellt und zerstört (default).
- *class*: die Fixture wird zerstört, nachdem die letzte Testfunktion einer Klasse aufgerufen worden ist.
- *module*: die Fixture wird nach der letzten Funktion eines Moduls zerstört.
- *package*: die Fixture wird nach der letzten Funktion eines Pakets zerstört.
- *session*: die Fixture wird am Ende der Session zerstört.

Wird in einem Testmodul eine Datenbank-Verbindung verwendet, dann kann diese beispielsweise in einer Fixture `@pytest.fixture(scope="module")` erzeugt werden.

Werden die Fixtures eines Projekts in ein Modul mit dem Namen `conftest.py` verschoben, werden sie von *pytest* automatisch gefunden, müssen also nicht explizit in das Testmodul importiert werden.

(siehe [pytest fixtures: explicit, modular, scalable](https://docs.pytest.org/en/stable/fixture.html))

### *pytest* plugins

Zu *pytest* gibt es eine grosse Anzahl von Plugins. Diese können mit *pip* installiert werden.

Mit dem Plugin [*pytest-cov*](https://github.com/pytest-dev/pytest-cov) kann beispielsweise die Testabdeckung des Codes bestimmt werden.

## Das *unittest* Modul:

Die Testklasse leitet von `unittest.TestCase` ab. Mit der `unittest.assertEqual()`-Methoden wird der aktuelle Wert mit einem erwarteten Wert verglichen. Weitere `.assert*()`-Methoden: `.assertTrue(x)`, `.assertFalse(x)`, `.assertIs(a, b)`, `.assertIsNone(x)`, `.assertIn(a, b)`, `.assertIsInstance(a, b)`.

### Beispiel: `SurveyResponse` Klasse

In [ ]:
%%writefile survey_response.py
from dataclasses import dataclass

@dataclass
class SurveyResponse:
    first_name: str = ""
    name: str = ""
    mail: str = ""
    lecture: str = ""
    kind: str = ""
    with_pwd: bool = False
    
    @classmethod
    def from_body(cls, body: str):
        lines = filter(None, body.split("\n\r"))
        entry_lines = (line.split(":", 1) for line in lines if ":" in line)
        entries = {key.strip(): val.strip() for key, val in entry_lines}
        key_mapping = {"First name": "name", "Vorname": "name"}
        entries = {key_mapping.get(key, key): val for key, val in entries.items()}
        return cls(**entries)

    def to_csv(self):
        return f"{self.kind}; {self.first_name}; {self.name}; {self.mail}; {self.lecture}; 'Nein'; "

In [ ]:
import unittest

from survey_response import SurveyResponse

class TestSurveyResponse(unittest.TestCase):
    """Test suite for SurveyResponse"""
    def setUp(self):
        self.body = """First name: Chuck
name: Norris
mail: chucknorris@roundhouse.gov
lecture: Roundhouse Kicks
"""
        self.response = SurveyResponse(
            first_name="Chuck",
            name="Norris",
            lecture="Roundhouse Kicks",
            mail="chucknorris@roundhouse.gov"
        )
        
    def tearDown(self):
        """Not needed here."""
    
    def test_survey_response_can_load_simple_response(self):
        expected_response = self.response
        self.assertEqual(SurveyResponse.from_body(self.body), expected_response)
        
    def test_survey_response_raises_error_for_invalid_field(self):
        body = self.body + "comment: Chucknorris counted to ∞ - twice!\n"
        with self.assertRaises(TypeError):
            SurveyResponse.from_body(body)
        
    def test_servey_response_creates_correct_csv_line(self):
        csv_line = self.response.to_csv()
        expected_csv_line = "; Chuck; Norris; chucknorris@roundhouse.gov; Roundhouse Kicks; 'Nein'; "
        self.assertEqual(csv_line, expected_csv_line)        

unittest.main(argv=[""], verbosity=2, exit=False)


## Testen mit *doctest*:

Sogenannte *doctests* sind eine elegante Methode, Tests und Dokumentation gleichzeitig zu erledigen (viele fortgeschrittene Progammierer betrachten alle Tests als Dokumentation).
Bei `doctest` wird eine Testanweisung im *docstring* definiert (mit `>>>`) und ausgewertet.

In [ ]:
from dataclasses import dataclass
import doctest

@dataclass
class SurveyResponse:
    first_name: str = ""
    name: str = ""
    mail: str = ""
    lecture: str = ""
    kind: str = ""
    with_pwd: bool = False
    
    @classmethod
    def from_body(cls, body: str):
        r"""
        >>> SurveyResponse.from_body("name: Norris\nfirst_name: Chuck")
        SurveyResponse(first_name='Chuck', name='Norris', mail='', lecture='', kind='', with_pwd=False)
        """
        lines = filter(None, body.split("\n\r"))
        entry_lines = (line.split(":", 1) for line in lines if ":" in line)
        entries = {key.strip(): val.strip() for key, val in entry_lines}
        key_mapping = {"First name": "name", "Vorname": "name"}
        entries = {key_mapping.get(key, key): val for key, val in entries.items()}
        return cls(**entries)

    def to_csv(self):
        """
        >>> SurveyResponse(first_name="Chuck", name="Norris").to_csv()
        "; Chuck; Norris; ; ; 'Nein'; "
        """
        return f"{self.kind}; {self.first_name}; {self.name}; {self.mail}; {self.lecture}; 'Nein'; "
    
doctest.testmod()

## Exercise 1
We review the example of the previous section about installing software on servers.
Start with the following:

In [ ]:
%%writefile server.py

class Server:
    def __init__(self):
        self.installed = []
    
    def install(self, software):
        self.installed.append(software)
        return len(self.installed)

Write unit tests which check the following:
- [ ] A new instance of Server has no installed software:
  - **Given**: A new instance of `Server`,
  - **When**: -
  - **Then**: The instance has an empty `installed` attribute.
- [ ] A new server has one installed software after a call to `server.install()`:
  - **Given**: A new instance of `Server`,
  - **When**: `server.install("firefox")` is called,
  - **Then**: The instance has `installed = ["firefox"]`.
- [ ] A new server has 2 installed software packages after two calls to `server.install()`:
  - **Given**: A new instance of `Server`,
  - **When**: `server.install("firefox")` and `server.install("python")` is called,
  - **Then**: The instance has `"firefox"` and `"python"` in `installed`.

## Solution (Exercise 1)

In [ ]:
import server

In [ ]:
%%pytest

import pytest

from server import Server

def test_new_server_has_no_installed_software():
    server = Server()
    assert not server.installed

def test_new_server_has_one_install_after_call_to_install():
    server = Server()
    server.install("firefox")
    assert server.installed == ["firefox"]

def test_new_server_has_2_installs_after_2_calls_to_install():
    server = Server()
    server.install("firefox")
    server.install("python")
    assert set(server.installed) == {"firefox", "python"}

## Exercise 2
We will test a modified version of command line interface `iam_example.py` from section 3.
To test the command line interface, we add an `*args` argument to the `main(*args)` function and pass it on to `parser.parse_args(*args)`. If `*args` is empty (e.g. when we use the CLI), then `parser.parse_args()` will be called without arguments and thus automatically parse the arguments on the command line (`sys.argv`) instead. If `*args`  is *not* empty (e.g. when we run a test), then `parser.parse_args(*args)` will parse our test-arguments:

```python
def main(*args):
    ...
    args = parser.parse_args(*args)
    ...
```

This will allow us to call for example
```python
main(["user", "--add", "chucknorris"])
```
Which corresponds to the following prompt from the command line:
```sh
./iam_example.py user --add chucknorris
```

In [ ]:
%%writefile iam_example.py
#!/bin/env python3

import argparse

class Database:
    def __init__(self, users=None):
        self.users = users if users is not None else set()

    def add_user(self, user):
        self.users.add(user)

    def remove_user(self, user):
        try:
            self.users.remove(user)
        except KeyError:
            raise KeyError(f"No such user: {user}") from None

def main(db, *args):
    parser = argparse.ArgumentParser(description="IAM tool")
    subparsers = parser.add_subparsers(
        dest="subcmd",
        required=True,
        title="List of sub-commands",
        description="For an overview of action specific parameters, use %(prog)s <SUB-COMMAND> --help",
        help="Sub-command help", metavar="<SUB-COMMAND>"
    )
    subparser_user = subparsers.add_parser("user", help="Manage users")
    subparser_user.add_argument("--add", help="Add a user")
    subparser_user.add_argument("--rm", help="Remove a user")

    args = parser.parse_args(*args)
    if args.subcmd == "user":
        if args.add:
            db.add_user(args.add)
        if args.rm:
            try:
                db.remove_user(args.rm)
            except KeyError as e:
                raise SystemExit("💥 " + e.args[0]) from e

if __name__ == "__main__":
    db = Database()
    main(db)

Write unit tests for the following cases:
- [ ] `./iam_example.py user --add chucknorris` adds a user `"chucknorris"`.
      Write the test by actually testing:

  - **Given**: An empty database `db`,
  - **When**: Calling `main(db, ...)`, where `...` represents the list of arguments corresponding to the above call,
  - **Then**: `db.users` is equal to `{"chucknorris"}`.
- [ ] `./iam_example.py user --rm user` removes a user `"user"` if that user existed:
  - **Given**: An database `db` with one user `"user"`,
  - **When**: Calling `main(db, ...)`, where `...` represents the list of arguments corresponding to the above call,
  - **Then**: `db.users` is empty.
- [ ] `./iam_example.py user --rm user2` exits the program with an error message if the user did not exist:
  - **Given**: An database `db` with one user `"user"`,
  - **When**: Calling `main(db, ...)`, where `...` represents the list of arguments corresponding to the above call,
  - **Then**: An exception of type `SystemExit` is raised, which contains a message `"💥 No such user: user2"`.

  Hint: Use [with pytest.raises()](https://docs.pytest.org/en/stable/reference/reference.html#pytest-raises) or [with unittest.TestCase.assertRaises()](https://docs.python.org/3/library/unittest.html#unittest.TestCase.assertRaises)!

## Solution (Exercise 2)

In [ ]:
import iam_example

In [ ]:
%%pytest

import pytest

from iam_example import main, Database

def test_add_user_works():
    db = Database()
    main(db, ["user", "--add", "chucknorris"])
    assert db.users == {"chucknorris"}

def test_remove_user_works_for_existing_user():
    db = Database(users={"user"})
    main(db, ["user", "--rm", "user"])
    assert not db.users

def test_remove_user_gives_an_error_for_non_existing_user():
    db = Database(users={"user"})
    with pytest.raises(SystemExit, match="💥 No such user: user2") as e_info:
        main(db, ["user", "--rm", "user2"])
